In [384]:
import pandas as pd

In [385]:
teamList = page2.findAll('div', {'class': 'teamname'})
urls = [team.find('a')['href'] for team in teamList]
teams = [team.text.replace('\n', '') for team in teamList]
teamsDict = dict(zip(urls, teams))
# only have data starting in 2018. before that is behind a pay wall
years = list(range(2018, 2027, 1))

In [386]:
appended = pd.DataFrame()
for year in years: 
    for team in teamsDict: 
        html = team+f'{year}/'
        tables = pd.read_html(html)
        print(f'Pulling data from {html}, Team: {teamsDict.get(team)}, Year: {year}')
        for table in tables[:-1]: 
            # change 1st column name to Player
            firstColumn = table.columns[0]
            table = table.rename(columns = {firstColumn: 'Player'})
            # add a column for year
            table['Year'] = str(year)
            # add a column for team
            table['Team'] = teamsDict.get(team)
            # add a column for cap type
            capType = table.iloc[-1,:][0].split(':')[0]
            table['Cap Type'] = capType
            # drop the last row
            table = table[:-1]
            # append to final df. ignoring some specific tables with mismatching column types
            if capType != 'Projected Draft Pool': 
                appended = appended.append(table)

Pulling data from https://www.spotrac.com/nfl/arizona-cardinals/cap/2018/, Team: Arizona Cardinals, Year: 2018
Pulling data from https://www.spotrac.com/nfl/atlanta-falcons/cap/2018/, Team: Atlanta Falcons, Year: 2018
Pulling data from https://www.spotrac.com/nfl/baltimore-ravens/cap/2018/, Team: Baltimore Ravens, Year: 2018
Pulling data from https://www.spotrac.com/nfl/buffalo-bills/cap/2018/, Team: Buffalo Bills, Year: 2018
Pulling data from https://www.spotrac.com/nfl/carolina-panthers/cap/2018/, Team: Carolina Panthers, Year: 2018
Pulling data from https://www.spotrac.com/nfl/chicago-bears/cap/2018/, Team: Chicago Bears, Year: 2018
Pulling data from https://www.spotrac.com/nfl/cincinnati-bengals/cap/2018/, Team: Cincinnati Bengals, Year: 2018
Pulling data from https://www.spotrac.com/nfl/cleveland-browns/cap/2018/, Team: Cleveland Browns, Year: 2018
Pulling data from https://www.spotrac.com/nfl/dallas-cowboys/cap/2018/, Team: Dallas Cowboys, Year: 2018
Pulling data from https://www

In [387]:
# Reset index
final = appended.reset_index(drop=True)
# Drop unnamed column
final = final.drop(columns=['Unnamed: 12'])
# Reorder cols 
cols = ['Team', 'Cap Type', 'Year', 'Player', 'Pos.', 'Base Salary', 'Signing Bonus', 'Roster Bonus', 'Option Bonus',
        'Workout Bonus', 'Restruc. Bonus', 'Misc.', 'Dead Cap', 'Cap Hit', 'Cap %']
final[cols].head()

,Team,Cap Type,Year,Player,Pos.,Base Salary,Signing Bonus,Roster Bonus,Option Bonus,Workout Bonus,Restruc. Bonus,Misc.,Dead Cap,Cap Hit,Cap %
0,Arizona Cardinals,Active Roster,2018,Larry Fitzgerald,WR,"$11,000,000","$2,500,000",-,"$2,350,000",-,-,"$1,000,000","($15,850,000)","$16,850,000",9.42
1,Arizona Cardinals,Active Roster,2018,Chandler Jones,DE,"$12,500,000","$3,000,000",-,-,-,-,-,"($36,000,000)","$15,500,000",8.67
2,Arizona Cardinals,Active Roster,2018,Patrick Peterson,CB,"$11,000,000","$3,072,377",-,-,"$250,000","$634,588",-,"($16,226,141)","$14,956,965",8.37
3,Arizona Cardinals,Active Roster,2018,Deone Bucannon,OLB,"$8,718,000",-,-,-,-,-,-,"($8,718,000)","$8,718,000",4.88
4,Arizona Cardinals,Active Roster,2018,Jermaine Gresham,TE,"$5,250,000","$1,750,000","$218,750",-,-,-,-,"($12,750,000)","$7,218,750",4.04


In [388]:
final.shape

(15351, 16)

In [389]:
final.to_csv('NFL_Salary_Cap_2018_2026.csv')